In [1]:
import pandas as pd
import requests
import plotly.express as px
import os


In [2]:
headers = {
'accept': 'application/json',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
'authorization': f"Token {os.environ['GUINNESS_TOKEN']}",
'cache-control': 'no-cache',
'content-length': '37',
'content-type': 'application/json',
# cookie: _ga=GA1.2.1540116150.1676049895; _gid=GA1.2.286835348.1676049895; _fbp=fb.1.1676049895499.412884114; _ga_Z8CQEC7B8T=GS1.1.1676049894.1.1.1676049904.0.0.0; _ga=GA1.3.1540116150.1676049895; _gid=GA1.3.286835348.1676049895; _gat_tracker1=1; _gat_tracker2=1
'origin': 'https://fantasy.sixnationsrugby.com',
'pragma': 'no-cache',
'referer': 'https://fantasy.sixnationsrugby.com/',
'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
'sec-ch-ua-mobile': '?0',
'sec-ch-ua-platform': '"macOS"',
'sec-fetch-dest': 'empty',
'sec-fetch-mode': 'cors',
'sec-fetch-site': 'same-origin',
'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
'x-access-key': '600@14.05@',
    
}
stats_url = 'https://fantasy.sixnationsrugby.com/v1/private/statsjoueur'
player_url = 'https://fantasy.sixnationsrugby.com/v1/private/searchjoueurs'

params={'lg':'en'}

week = 3

# {"filters":{"nom":"","club":"","position":"","budget_ok":false,"engage":false,"partant":false,"dreamteam":false,"quota":"","idj":"2","pageIndex":3,"pageSize":10,"loadSelect":0,"searchonly":1}}

def get_player_stats(player_id):
    json = {"credentials":{"idj": "2", "idf": player_id}}
    r = requests.post(stats_url, json=json, headers=headers, params=params)
    return r.json()

def get_players(page):
    json_players = {"filters":{"nom":"","club":"","position":"","budget_ok":False,"engage":'false',"partant":False,"dreamteam":False,"quota":"","idj":f"{week}","pageIndex":page,"pageSize":10,"loadSelect":0,"searchonly":1}}
    r = requests.post(player_url, headers=headers, json=json_players, params=params)
    return r

def build_players_df(num_pages):
    player_list = []
    for i in range(0, num_pages):
        try:
            player_response = get_players(i)
            players = [(j['id'], 
                        j['nomcomplet'], 
                        j['club'], 
                        j['id_position'],
                       ) for j in player_response.json()['joueurs']]
            for p in players:
                player_list.append(p)
        except Exception as e:
            print(e)
    out_df = pd.DataFrame(player_list)
    out_df.columns = ['id', 'name', 'team', 'position_id']
    return out_df

def process_stats(json, player_id):
    out_dict = {}
    out_dict['id'] = player_id
    for r in json['points_marques']:
        out_dict[f"points_round_{r['numero']}"] = r['nb_points']
    for i,r in enumerate(json['valeur_footballeur']):
        out_dict[f"price_round_{i+1}"] = r
    for stat in json['stats_individuelles']:
        stat_name = stat_name_dict[stat['libelle']]
        out_dict[stat_name] = stat['total']
        out_dict['num_matches'] = stat['nbmatchs']
    return out_dict

def build_player_stats(players_df):
    player_stats = []
    for i,p in players_df.iterrows():
        try:
            stat = get_player_stats(p['id'])
            player_stats.append(process_stats(stat, p['id']))
        except Exception as e:
            print(e)
    stats_df = pd.DataFrame(player_stats)
    return players_df.merge(stats_df, on='id').set_index('name').fillna(0)


stat_name_dict = {
    'Man of the match':'Man of the match',
    'Tackles':'Tackles',
    'Dominant tackles':'Dominant tackles',
    'Line-breaks':'Line-breaks',
    'Metres carried':'Metres carried',
    'Kick 50-22':'Kick 50-22',
    'Lineout steal':'Lineout steal',
    'Breakdown steal':'Breakdown steal',
    'Try':'Try',
    'Assists':'Assists',
    'Conversion':'Conversion',
    'Penalty':'Penalty',
    'Drop goal':'Drop goal',
    'Yellow cards':'Yellow cards',
    'Red cards ':'Red cards ',
    
    'Uomo partita':'Man of the match', 
    'Placcaggi':'Tackles', 
    'Placcaggi dominanti':'Dominant tackles', 
    'Line-breaks*':'Line-breaks', 
    'Metri palla in mano':'Metres carried', 
    '50-22':'Kick 50-22', 
    'Lineout steal*':'Lineout steal', 
    'Palla rubata al breakdown':'Breakdown steal', 
    'Meta':'Try', 
    'Assists*': 'Assists', 
    'Trasformazione': 'Conversion', 
    'Punizione':'Penalty', 
    'Drop':'Drop goal', 
    'Cartellini gialli':'Yellow cards', 
    'Cartellini rossi':'Red cards ', 
    
    'Homme du match':'Man of the match', 
    'Plaquages':'Tackles', 
    'Plaquages offensifs':'Dominant tackles', 
    'Franchissement':'Line-breaks', 
    'Mètres avec le ballon':'Metres carried', 
    'Dégagement 50-22':'Kick 50-22', 
    'Interception en touche':'Lineout steal', 
    'Interception au sol':'Breakdown steal', 
    'Essai':'Try', 
    'Passes décisives':'Assists', 
    'Transformation':'Conversion', 
    'Pénalité':'Penalty', 
    'Cartons jaunes':'Yellow cards', 
    'Cartons rouges':'Red cards '
    
}

position_lookup = {
    6: 'back-three',
    7:'centres',
    8:'fly-half',
    9:'scrum-half',
    10:'back-row',
    11:'second-row',
    12:'front-row',
    13:'hooker'
}

players = build_players_df(250)
stats = build_player_stats(players)
stats['position'] = stats['position_id'].map(lambda x: position_lookup[x])


In [3]:
rounds = [1,2]
for round in rounds:
    stats[f'price_round_{round}'] = pd.to_numeric(stats[f'price_round_{round}'])
    stats[f'points_round_{round}'] = pd.to_numeric(stats[f'points_round_{round}'])
    stats[f'played_round_{round}'] = stats[f'points_round_{round}'] != 0
    stats[f'points_to_price_round_{round}'] = stats[f'points_round_{round}'] / stats[f'price_round_{round}']
    
stats['latest_price'] = stats[f'price_round_{rounds[-1]}']
stats['total_points_all_rounds'] = stats.filter(like='points_round').sum(axis=1)
stats['points_per_round_all_rounds'] = stats['total_points_all_rounds'] / stats['num_matches']
stats['total_points_to_latest_price_ratio'] = stats['points_per_round_all_rounds'] / stats['latest_price']


stats.to_csv('data/data.csv')